# Despacho múltiplo

Neste notebook, vamos explorar o conceito de [despacho múltiplo](https://en.wikipedia.org/wiki/Multiple_dispatch), que é um recurso chave da linguagem Julia.

Para entender despacho múltiplo em Julia, vamos começar com um exemplo que nós já vimos.

Em Julia, podemos declarar funções sem informar o tipo dos argumentos da função.
Por exemplo:

In [1]:
f(x) = x^2

f (generic function with 1 method)

Então a Julia vai determinar automaticamente sobre quais tipos de argumentos faz sentido aplicar a função, e sobre quais tipos não faz:

In [2]:
f(9)

81

In [3]:
f([1, 2, 3])

LoadError: MethodError: no method matching ^(::Vector{Int64}, ::Int64)
[0mClosest candidates are:
[0m  ^([91m::Union{AbstractChar, AbstractString}[39m, ::Integer) at strings/basic.jl:718
[0m  ^([91m::Complex{var"#s79"} where var"#s79"<:AbstractFloat[39m, ::Integer) at complex.jl:818
[0m  ^([91m::Complex{var"#s79"} where var"#s79"<:Integer[39m, ::Integer) at complex.jl:820
[0m  ...

## Especificando os tipos dos argumentos

Temos a opção de declarar explicitamente quais tipos de argumentos uma função pode receber.
Por exemplo, vamos definir uma função `teste` que só pode receber argumentos do tipo `String`:

In [4]:
teste(x::String, y::String) = println("Os argumentos x e y são strings")

teste (generic function with 1 method)

Vamos verificar que a função `teste` age sobre strings, mas não age sobre argumentos de outros tipos:

In [5]:
teste("despacho", "múltiplo")

Os argumentos x e y são strings


In [6]:
teste(2, 3)

LoadError: MethodError: no method matching teste(::Int64, ::Int64)

Para fazer com que a função `teste` aceite números inteiros, vamos definir `teste` para argumentos do tipo `Int`:

In [7]:
teste(x::Int, y::Int) = println("Os argumentos x e y são números inteiros")

teste (generic function with 2 methods)

In [8]:
teste(2, 3)

Os argumentos x e y são números inteiros


Agora `teste` age sobre os inteiros.
Mas observe, a função `teste` continua definida quando `x` e `y` são strings:

In [9]:
teste("despacho", "múltiplo")

Os argumentos x e y são strings


Quando declaramos
```julia
teste(x::Int, y::Int) = println("Os argumentos x e y são números inteiros"),
```
não removemos ou substituímos
```julia
teste(x::String, y::String),
```
simplesmente adicionamos um método à função genérica `teste`.

Uma **função genérica** é um conceito abstrato associado a uma operação.
Por exemplo, a função genérica `+` representa o conceito de adição.
Um **método** é uma implementação específica de uma função genérica para um tipo específico de argumento.
Por exemplo, a função `+` possui métodos para números de precisão flutuante, inteiros, matrizes, etc.
Podemos usar `methods` para ver os métodos que a função `teste` possui:

In [10]:
methods(teste)

# 2 methods for generic function "teste":
[1] teste(x::String, y::String) in Main at In[4]:1
[2] teste(x::Int64, y::Int64) in Main at In[7]:1

E podemos ver os métodos da função `+`:

In [11]:
methods(+)

# 190 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:528
[3] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:534
[4] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:376
[5] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:384
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:392
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /usr/share/julia/stdlib/v1.6/Dates/src/periods.jl:362
[8] +(A::LinearAlgebra.UpperHessenberg, B::LinearAlgebra.UpperHessenberg) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/hessenberg.jl:101
[9] +(H::LinearAlgebra.UpperHessenberg, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/hessenberg.jl:106
[10] +(x::P, y::P) where P<:Dates.Period in Dates at /usr/share/julia/stdlib/v1.6/Dates/src/periods.jl:79
[11] +(x::Dates.Period, y::Dates.Period) in Dates at /usr/share/julia/stdlib/v1.6/Dates/src/periods.jl:351
[12] +(y::Dates.Period, x::Dates.CompoundPeriod) in Dates at /usr/share/julia/stdlib/v1.6/Dates/src/periods.jl:353
[13] +(y::Dates.Period, x::Dates.TimeType) in Dates at /usr/share/julia/stdlib/v1.6/Dates/src/arithmetic.jl:85
[14] +(x::Dates.Period, r::AbstractRange{var"#s832"} where var"#s832"<:Dates.TimeType) in Dates at /usr/share/julia/stdlib/v1.6/Dates/src/ranges.jl:62
[15] +(y::Union{Dates.CompoundPeriod, Dates.Period}, x::AbstractArray{var"#s832", N} where {var"#s832"<:Dates.TimeType, N}) in Dates at /usr/share/julia/stdlib/v1.6/Dates/src/deprecated.jl:14
[16] +(z::Complex, w::Complex) in Base at complex.jl:275
[17] +(r1::LinRange{T}, r2::LinRange{T}) where T in Base at range.jl:1110
[18] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Tridiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/tridiag.jl:733
[19] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/special.jl:157
[20] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Diagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/special.jl:173
[21] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/special.jl:193
[22] +(A::LinearAlgebra.Tridiagonal{var"#s832", V} where {var"#s832"<:Number, V<:AbstractVector{var"#s832"}}, B::LinearAlgebra.UniformScaling) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/special.jl:226
[23] +(x::Base.TwicePrecision, y::Number) in Base at twiceprecision.jl:267
[24] +(x::Base.TwicePrecision{T}, y::Base.TwicePrecision{T}) where T in Base at twiceprecision.jl:273
[25] +(x::Base.TwicePrecision, y::Base.TwicePrecision) in Base at twiceprecision.jl:278
[26] +(r1::OrdinalRange, r2::OrdinalRange) in Base at range.jl:1103
[27] +(r::AbstractRange{var"#s832"} where var"#s832"<:Dates.TimeType, x::Dates.Period) in Dates at /usr/share/julia/stdlib/v1.6/Dates/src/ranges.jl:63
[28] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/bidiag.jl:354
[29] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.UpperTriangular) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/special.jl:94
[30] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.UnitUpperTriangular) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/special.jl:94
[31] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.LowerTriangular) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/special.jl:94
[32] +(A::LinearAlgebra.Bidiagonal, B::LinearAlgebra.UnitLowerTriangular) in LinearAlgebra at /usr/share/julia/stdlib/v1.6/LinearAlgebra/src/special.jl:94
[

Portanto, agora podemos aplicar `teste` a strings e inteiros.
Quando aplicamos `teste` a um par de argumentos, a Julia vai inferir os tipos dos argumentos e despachar o método apropriado.
Isso é **despacho múltiplo**.
Despacho múltiplo torna os programas genéricos e rápidos.

Para ver qual método está sendo despachado quando chamamos uma função genérica, usamos a macro `@which`:

In [12]:
@which teste(2, 3)

teste(x::Int64, y::Int64) in Main at In[7]:1

In [13]:
@which 3.0 + 5.0

+(x::Float64, y::Float64) in Base at float.jl:326